# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 31 2022 8:47AM,249662,10,0086128112,ISDIN Corporation,Released
1,Oct 31 2022 8:47AM,249662,10,0086128114,ISDIN Corporation,Released
2,Oct 31 2022 8:47AM,249662,10,0086128115,ISDIN Corporation,Released
3,Oct 31 2022 8:47AM,249662,10,0086128116,ISDIN Corporation,Released
4,Oct 31 2022 8:47AM,249662,10,0086128121,ISDIN Corporation,Released
5,Oct 31 2022 8:47AM,249662,10,0086128123,ISDIN Corporation,Released
6,Oct 31 2022 8:47AM,249662,10,0086128119,ISDIN Corporation,Released
7,Oct 31 2022 8:47AM,249662,10,0086128125,ISDIN Corporation,Released
8,Oct 31 2022 8:47AM,249662,10,0086128128,ISDIN Corporation,Released
9,Oct 31 2022 8:47AM,249662,10,0086128130,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
11,249657,Released,56
12,249658,Released,23
13,249659,Released,43
14,249660,Released,46
15,249662,Released,62


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
249657,NaN,56.0
249658,NaN,23.0
249659,NaN,43.0
249660,NaN,46.0
249662,NaN,62.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
249646,0.0,9.0
249647,7.0,3.0
249649,0.0,4.0
249650,0.0,1.0
249651,0.0,18.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
249646,0,9
249647,7,3
249649,0,4
249650,0,1
249651,0,18


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,249646,0,9
1,249647,7,3
2,249649,0,4
3,249650,0,1
4,249651,0,18


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,249646,,9
1,249647,7,3
2,249649,,4
3,249650,,1
4,249651,,18


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 31 2022 8:47AM,249662,10,ISDIN Corporation
62,Oct 31 2022 8:47AM,249660,10,ISDIN Corporation
108,Oct 31 2022 8:47AM,249659,10,ISDIN Corporation
151,Oct 31 2022 8:47AM,249658,10,ISDIN Corporation
174,Oct 31 2022 8:47AM,249657,10,ISDIN Corporation
230,Oct 31 2022 8:47AM,249656,10,ISDIN Corporation
232,Oct 31 2022 8:47AM,249655,10,ISDIN Corporation
234,Oct 31 2022 8:47AM,249654,10,ISDIN Corporation
241,Oct 31 2022 8:37AM,249653,10,Bio-PRF
242,Oct 31 2022 8:27AM,249652,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Oct 31 2022 8:47AM,249662,10,ISDIN Corporation,,62
1,Oct 31 2022 8:47AM,249660,10,ISDIN Corporation,,46
2,Oct 31 2022 8:47AM,249659,10,ISDIN Corporation,,43
3,Oct 31 2022 8:47AM,249658,10,ISDIN Corporation,,23
4,Oct 31 2022 8:47AM,249657,10,ISDIN Corporation,,56
5,Oct 31 2022 8:47AM,249656,10,ISDIN Corporation,,2
6,Oct 31 2022 8:47AM,249655,10,ISDIN Corporation,,2
7,Oct 31 2022 8:47AM,249654,10,ISDIN Corporation,,7
8,Oct 31 2022 8:37AM,249653,10,Bio-PRF,,1
9,Oct 31 2022 8:27AM,249652,22,"NBTY Global, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 31 2022 8:47AM,249662,10,ISDIN Corporation,62,
1,Oct 31 2022 8:47AM,249660,10,ISDIN Corporation,46,
2,Oct 31 2022 8:47AM,249659,10,ISDIN Corporation,43,
3,Oct 31 2022 8:47AM,249658,10,ISDIN Corporation,23,
4,Oct 31 2022 8:47AM,249657,10,ISDIN Corporation,56,
5,Oct 31 2022 8:47AM,249656,10,ISDIN Corporation,2,
6,Oct 31 2022 8:47AM,249655,10,ISDIN Corporation,2,
7,Oct 31 2022 8:47AM,249654,10,ISDIN Corporation,7,
8,Oct 31 2022 8:37AM,249653,10,Bio-PRF,1,
9,Oct 31 2022 8:27AM,249652,22,"NBTY Global, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 31 2022 8:47AM,249662,10,ISDIN Corporation,62,
1,Oct 31 2022 8:47AM,249660,10,ISDIN Corporation,46,
2,Oct 31 2022 8:47AM,249659,10,ISDIN Corporation,43,
3,Oct 31 2022 8:47AM,249658,10,ISDIN Corporation,23,
4,Oct 31 2022 8:47AM,249657,10,ISDIN Corporation,56,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Oct 31 2022 8:47AM,249662,10,ISDIN Corporation,62,NaN
1,Oct 31 2022 8:47AM,249660,10,ISDIN Corporation,46,NaN
2,Oct 31 2022 8:47AM,249659,10,ISDIN Corporation,43,NaN
3,Oct 31 2022 8:47AM,249658,10,ISDIN Corporation,23,NaN
4,Oct 31 2022 8:47AM,249657,10,ISDIN Corporation,56,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 31 2022 8:47AM,249662,10,ISDIN Corporation,62,0.0
1,Oct 31 2022 8:47AM,249660,10,ISDIN Corporation,46,0.0
2,Oct 31 2022 8:47AM,249659,10,ISDIN Corporation,43,0.0
3,Oct 31 2022 8:47AM,249658,10,ISDIN Corporation,23,0.0
4,Oct 31 2022 8:47AM,249657,10,ISDIN Corporation,56,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2995864,265,0.0
19,499293,12,7.0
22,249652,1,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2995864,265,0.0
1,19,499293,12,7.0
2,22,249652,1,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,265,0.0
1,19,12,7.0
2,22,1,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,265.0
1,19,Released,12.0
2,22,Released,1.0
3,10,Executing,0.0
4,19,Executing,7.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,19,22
Status,,,
Executing,0.0,7.0,0.0
Released,265.0,12.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,19,22
0,Executing,0.0,7.0,0.0
1,Released,265.0,12.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,19,22
0,Executing,0.0,7.0,0.0
1,Released,265.0,12.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()